## Namewrapper Fuse Statistics

### Todo

[ X ] Get wrapped names

[ X ] Get wrapped names with fuses set

[ X ] Get wrapped names that unwrapped

[ X ] Determine how many names are locked (PCC & Cannot Unwrap = True)


In [1]:
# Imports
from web3 import Web3
from concurrent.futures import ThreadPoolExecutor, as_completed
from hexbytes import HexBytes
import asyncio
import nest_asyncio
import pandas as pd
import json
nest_asyncio.apply()
pd.set_option('future.no_silent_downcasting', True)

In [2]:
RPC = "http://127.0.0.1:8545"

In [3]:
# Connect to RPC
web3 = Web3(Web3.HTTPProvider(RPC))
assert web3.is_connected()

In [4]:
async def fetch_logs_for_range(web3, address, start_block, end_block):
    """Fetch logs for a specific block range asynchronously."""
    try:
        events = await asyncio.to_thread(
            web3.eth.get_logs, {
                "fromBlock": start_block,
                "toBlock": end_block,
                "address": address
            }
        )
        print(f"Fetched {len(events)} events from blocks {start_block} to {end_block}")
        return events
    except Exception as e:
        print(f"Error fetching logs for blocks {start_block} to {end_block}: {e}")
        return []

async def fetch_logs_parallel(web3, address, start_block, end_block, step=100000, max_concurrent_tasks=10):
    """Fetch logs in parallel using asyncio for better scalability."""
    tasks = []
    logs = []

    print(f"Fetching events for {address} from block {start_block} to {end_block} in parallel...")

    semaphore = asyncio.Semaphore(max_concurrent_tasks)

    async def fetch_with_semaphore(block_start, block_end):
        async with semaphore:
            return await fetch_logs_for_range(web3, address, block_start, block_end)

    for block_start in range(start_block, end_block + 1, step):
        block_end = min(block_start + step - 1, end_block)
        tasks.append(fetch_with_semaphore(block_start, block_end))

    results = await asyncio.gather(*tasks)
    for result in results:
        logs.extend(result)

    print(f"Total events found: {len(logs)}")
    return logs

In [5]:
def decode_dns_name(data, offset=0):
    """
    Decode a DNS name from a byte sequence.

    :param data: Byte sequence containing the DNS name.
    :param offset: Starting offset for the DNS name in the byte sequence.
    :return: The decoded DNS name as a string.
    """
    labels = []
    while True:
        length = data[offset]
        if length == 0:
            # End of the name
            break
        if (length & 0xC0) == 0xC0:
            # Pointer to another location
            pointer = ((length & 0x3F) << 8) | data[offset + 1]
            labels.append(decode_dns_name(data, pointer)[0])
            offset += 2
            break
        else:
            # Regular label
            offset += 1
            labels.append(data[offset:offset + length].decode("utf-8"))
            offset += length
    return ".".join(labels), offset + 1

In [6]:
CONTRACT_ADDRESS = "0xD4416b13d2b3a9aBae7AcD5D6C2BbDBE25686401" # NameWrapper
START_BLOCK = 16925608  
END_BLOCK = web3.eth.block_number  

In [7]:
logs = await fetch_logs_parallel(web3, CONTRACT_ADDRESS, START_BLOCK, END_BLOCK)


Fetching events for 0xD4416b13d2b3a9aBae7AcD5D6C2BbDBE25686401 from block 16925608 to 21332957 in parallel...
Fetched 145 events from blocks 16925608 to 17025607
Fetched 4678 events from blocks 17025608 to 17125607
Fetched 12087 events from blocks 17125608 to 17225607
Fetched 16522 events from blocks 17225608 to 17325607
Fetched 21250 events from blocks 17325608 to 17425607
Fetched 25262 events from blocks 17625608 to 17725607
Fetched 28513 events from blocks 17725608 to 17825607
Fetched 30687 events from blocks 17825608 to 17925607
Fetched 38019 events from blocks 17525608 to 17625607
Fetched 24615 events from blocks 18025608 to 18125607
Fetched 41900 events from blocks 17425608 to 17525607
Fetched 27044 events from blocks 17925608 to 18025607
Fetched 27258 events from blocks 18125608 to 18225607
Fetched 18170 events from blocks 18525608 to 18625607
Fetched 24724 events from blocks 18425608 to 18525607
Fetched 11826 events from blocks 18725608 to 18825607
Fetched 36004 events from blo

In [8]:
df = pd.DataFrame(logs)

In [9]:
topics = {'NameUnwrapped':'0xee2ba1195c65bcf218a83d874335c6bf9d9067b4c672f3c3bf16cf40de7586c4',
          'NameWrapped':'0x8ce7013e8abebc55c3890a68f5a27c67c3f7efa64e584de5fb22363c606fd340',
          'FusesSet':'0x39873f00c80f4f94b7bd1594aebcf650f003545b74824d57ddf4939e3ff3a34b'}

In [10]:
df['nameunwrap_flag'] = df['topics'].apply(
    lambda topic_list: any(t == HexBytes(topics['NameUnwrapped']) for t in topic_list)
)

df['namewrap_flag'] = df['topics'].apply(
    lambda topic_list: any(t == HexBytes(topics['NameWrapped']) for t in topic_list)
)

df['fusesset_flag'] = df['topics'].apply(
    lambda topic_list: any(t == HexBytes(topics['FusesSet']) for t in topic_list)
)

In [11]:
fuses_set_df = df[df.fusesset_flag == True].copy()

In [12]:
fuses_set_df['node'] = fuses_set_df['topics'].apply(lambda x: x[1] if len(x) > 1 else None)
fuses_set_df['fuses'] = fuses_set_df['data'].apply(lambda x: x.hex())

In [13]:
# Keep only latest fuse set
fuses_set_df = fuses_set_df.sort_values(by=['blockNumber', 'transactionIndex'], ascending=[False, False])
fuses_set_df = fuses_set_df.drop_duplicates(subset='node', keep='first')

In [14]:
# Decode fuses
FUSES = {
    "CANNOT_UNWRAP": 1,
    "CANNOT_BURN_FUSES": 2,
    "CANNOT_TRANSFER": 4,
    "CANNOT_SET_RESOLVER": 8,
    "CANNOT_SET_TTL": 16,
    "CANNOT_CREATE_SUBDOMAIN": 32,
    "CANNOT_APPROVE": 64,
    "PARENT_CANNOT_CONTROL": 1 << 16,
    "IS_DOT_ETH": 1 << 17,
    "CAN_EXTEND_EXPIRY": 1 << 18,
    "CAN_DO_EVERYTHING": 0,
    "PARENT_CONTROLLED_FUSES": 0xFFFF0000,
    "USER_SETTABLE_FUSES": 0xFFFDFFFF,
}

def decode_flags(value):
    """Decode a hex value into a list of fuse flags."""
    
    value = int(value, 16) # assumes value is a hex string
    # Decode the flags by checking bits
    return [name for name, bit in FUSES.items() if value & bit]

In [15]:
# Decode each fuse into a list of flags
decoded_fuses = fuses_set_df['fuses'].apply(decode_flags)

# Create a DataFrame where each flag is a column and the value is a boolean
decoded_fuses_df = decoded_fuses.apply(lambda flags: {flag: True for flag in flags}).apply(pd.Series)

# Merge the new columns back into the original DataFrame
fuses_set_df = pd.concat([fuses_set_df, decoded_fuses_df], axis=1)

# Display the updated DataFrame
fuses_set_df.fillna(False,inplace=True)


In [16]:

locked_fuses = (fuses_set_df.CANNOT_UNWRAP == True) & (fuses_set_df.PARENT_CANNOT_CONTROL == True) & (fuses_set_df.IS_DOT_ETH == True)

In [17]:
cols = [ 'blockNumber', 'blockTimestamp', 'transactionHash', 
        'node', 'fuses']

In [18]:
locked_names_df = fuses_set_df[locked_fuses][cols]

In [19]:
# namewrapper can pull name from node
with open("namewrapperabi.json", "r") as abi_file:
    contract_abi = json.load(abi_file)

contract = web3.eth.contract(address=CONTRACT_ADDRESS, abi=contract_abi)

In [20]:
locked_names_df['name'] =  locked_names_df['node'].apply(lambda x: contract.functions.names(x).call())
locked_names_df['name'] = locked_names_df['name'].apply(lambda x: decode_dns_name(x)[0])

In [32]:
def bytes_to_uint256(b):
    return int.from_bytes(b, 'big')

In [33]:
locked_names_df['owner'] =  locked_names_df['node'].apply(lambda x: contract.functions.ownerOf(bytes_to_uint256(x)).call())

In [36]:
locked_names_df.to_csv('locked_names.csv', index=False)

In [40]:
# Burn or dead
sum(locked_names_df.owner.isin(['0x0000000000000000000000000000000000000000','0x000000000000000000000000000000000000dEaD']))

252

In [41]:
# Unique owners of locked names
locked_names_df[~locked_names_df.owner.isin(['0x0000000000000000000000000000000000000000'
                             ,'0x000000000000000000000000000000000000dEaD'])].owner.nunique()

648

In [42]:
# Locked names
locked_names_df[~locked_names_df.owner.isin(['0x0000000000000000000000000000000000000000'
                             ,'0x000000000000000000000000000000000000dEaD'])].name.count()

np.int64(938)